In [1]:
from Bio import SeqIO
from collections import Counter
import pandas as pd 
import re
import requests
import json
import time
import glob
import pprint
import seaborn as sns
import pylab as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import pycountry 
import pycountry_convert as pc
pp = pprint.PrettyPrinter()
import numpy as np
from tabulate import tabulate
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

## 1. Download and export the scholarly raw data based on the pubmedIDs

In [2]:
# open list of unique IDs
df = pd.read_csv('PATH\data\merged-marine_sequence_data[OBIS_and_patents].csv').drop(columns = ['Unnamed: 0.1'])


C:\Users\pauld\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
len(set(df['Searched Organism']))

31578

In [12]:
len(set(df['GenBank organism']))

84431

### Publist

In [ ]:
publist = list(set(df['Pubmed'][df['Pubmed'].notna()]))

In [ ]:
len(publist)

In [ ]:
publist_clean = []

for pub in publist: 
    if '.' in str(pub): 
        publist_clean.append(str(pub).split('.')[0])
    elif ';' in str(pub): 
        publist_clean.append(str(pub).split(';')[0])    
    else: publist_clean.append(str(pub))


In [16]:
df[df['Pubmed'].isin(publist_clean)]

,Unnamed: 0,ANs,Pubmed,Origin,Geo_coordinates,Dates,GenBank organism,Searched Organism
0,0,EU645372,18560569.0,NaN,NaN,2016,Crypthelia trophostega,Crypthelia trophostega
1,1,EU645278,18560569.0,NaN,NaN,2008,Crypthelia trophostega,Crypthelia trophostega
2,2,EU645433,18560569.0,NaN,NaN,2008,Crypthelia trophostega,Crypthelia trophostega
87,87,M91276,1534411.0,NaN,NaN,1993,Evasterias troschelii,Evasterias troschelii
88,88,M91275,1534411.0,NaN,NaN,1993,Evasterias troschelii,Evasterias troschelii
...,...,...,...,...,...,...,...,...
36849829,42097900,AJ717613,15450929.0,Philippines,NaN,2009,Conus virgo,conus virgo
36849837,42097908,EU169206,18304695.0,NaN,NaN,2008,Conus virgo,conus virgo
36849845,42097916,AJ851182,15652641.0,Philippines,NaN,2005,Conus virgo,conus virgo
36849846,42097917,AJ851181,15652641.0,Philippines,NaN,2005,Conus virgo,conus virgo


In [37]:
data = json.dumps(publist_clean)

In [38]:
# I have taken out all the 'inlcude' now, but may want to re-integrate this for more speed

data_dict = []
url = 'https://api.lens.org/scholarly/search'


# request body with scroll time of 1 minute
request_body = '''{
  "query": {
      "terms":  {
            "pmid": %s
      }
  }
  , "size": 100
  , "scroll": "1m"
}''' % data
headers = {'Authorization': API_KEY, 'Content-Type': 'application/json'}

# Recursive function to scroll through paginated results
def scroll(scroll_id):
    # Change the request_body to prepare for next scroll api call
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s"}''' % (scroll_id)
        
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    # If rate-limited, wait for n seconds and proceed the same scroll id
    # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
    if response.status_code == requests.codes.too_many_requests:
        time.sleep(8)
        scroll(scroll_id)
    # If the response is not ok here, better to stop here and debug it
    
    elif response.status_code != requests.codes.ok:
        print(response.json())
        print('Äahh')
    # If the response is ok, do something with the response, take the new scroll id and iterate
    
    # leave the function if it returns empty (because then everything is downloaded already)
    #json = response.json()
    #if json['data'] == []:
    #    return
    
    else:
        json = response.json()
        if json.get('results') is not None and json['results'] > 0:
            scroll_id = json['scroll_id'] # Extract the new scroll id from response
            #print(json['data']) #DO something with your data
            data_dict.append(response.json())
            print(len(data_dict))
            scroll(scroll_id)
            
# start recursive scrolling
scroll(scroll_id=None)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235


In [40]:
# save it
with open(PATH\data\full-data_scholarly-lens_Genbank[OBIS_and_patents].json', 'w') as f:
    json.dump(data_dict, f)

## 2. Processing the data

In [2]:
# open list of unique IDs
df = pd.read_csv(PATH\data\merged-marine_sequence_data[OBIS_and_patents].csv').drop(columns = ['Unnamed: 0'])

# open scholarly dataaset
with open(PATH\data\full-data_scholarly-lens_Genbank[OBIS_and_patents].json') as json_string:
    data_dict = json.load(json_string)

C:\Users\pauld\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
for i in data_dict: 
    for k, v in i.items(): 
        if k == 'data': 
            for publication in v: 
                print(publication['year_published'])
                break
        

2017
2011
2005
2012
2005
2018
2007
2012
2015
2018
2012
2004
1988
2021
2008
2009
2005
2019
2017
2015
2009
2001
2012
2010
2007
2011
2011
1992
2020
2010
2001
2003
2007
2006
2007
2016
2008
2021
2007
1996
2017
2010
1997
1998
2005
2010
2021
2002
2021
2016
2001
2014
2003
2000
2008
1990
2019
2011
2005
2007
2006
2021
1999
2010
1999
2016
2011
1999
2021
2002
1990
1994
2006
2001
2022
2002
2014
2011
2008
2000
2010
2009
2014
1999
2014
2007
1997
2011
1988
2010
2017
2004
2015
2007
2014
2006
1991
2004
1997
2011
2019
1993
2009
2004
1996
1997
2001
1999
2012
1997
2006
2006
2021
2013
1988
2021
2019
2007
2010
2013
1999
2007
2010
2014
2018
1994
2019
2016
2009
2009
2014
1995
2015
2018
2022
2012
2020
2001
1993
2013
2003
1987
2001
2007
2016
2002
2011
1991
2002
2017
1995
2006
2013
2015
2005
2003
2005
2003
2003
2020
2012
2021
2008
2014
2011
2018
1993
2020
2003
2000
2007
2011
2010
2014
2012
2006
2004
2011
2012
2001
2009
2000
2009
2018
1986
2013
2015
2003
2014
2010
2011
2009
2014
2022
2012
2002
2003
2001
2013
2009


In [26]:
# to avoid wrong double pubmedIDs (if a paper has changed pubmed, or has two for some reason)
l2 = list(set(df['Pubmed'][~df['Pubmed'].isnull()]))


# read the raw scholarly data and prepare for merging with CoO dataset
pubmed_ids = []
titles = []
dates = []
lens_scholarly_id = []
affiliations = []
affiliations_ids = []
affiliations_countries = []
authors_lastname = []
authors_firstname = []
author_ids = []
first_author = []
last_author = []
nr_of_times_cited_science = []
nr_of_times_cited_patents = []

for i in data_dict: 
    for k, v in i.items(): 
        if k == 'data': 
            for publication in v: 
                nr = 0
                for e in publication['external_ids']:
                    for keys, values in e.items():
                        if values == 'pmid' and int(e['value']) in l2: 
                            nr = nr+1
                            if nr >= 2:
                                break

                            n = 1
                            if 'authors' in publication.keys(): 
                                nr_authors = len(publication['authors'])
                                for author in publication['authors']: # I need the authors, because I need their affiliation countries
                                    if n == 1: 
                                        first_author.append('first_author')
                                    else: 
                                        first_author.append('not first_author')
                                    if n == nr_authors: 
                                        last_author.append('last_author')#
                                    else: 
                                        last_author.append('not last_author')
                                    # pubmedid
                                    pubmed_ids.append(int(e['value']))
                                    titles.append(publication['title'])
                                    dates.append(publication['year_published'])
                                    lens_scholarly_id.append(publication['lens_id'])
                                    
                                    #citation nrs. 
                                    times_cited_science_placeholder = ''
                                    if 'scholarly_citations_count' in publication.keys():
                                            times_cited_science_placeholder = publication['scholarly_citations_count'] 
                                    nr_of_times_cited_science.append(times_cited_science_placeholder)
                                    
                                    #citation nrs. 
                                    times_cited_patents_placeholder = ''
                                    if 'patent_citations_count' in publication.keys():
                                            times_cited_patents_placeholder = publication['patent_citations_count'] 
                                    nr_of_times_cited_patents.append(times_cited_patents_placeholder)
                                    
                                    
                                    #authors
                                    author_placeholder_firstname = ''
                                    author_placeholder_lastname = ''
                                    if 'first_name' in author.keys():                                    
                                        author_placeholder_firstname = author['first_name']
                                    if 'last_name' in author.keys():
                                        author_placeholder_lastname = author['last_name']
                                    authors_firstname.append(author_placeholder_firstname)
                                    authors_lastname.append(author_placeholder_lastname)

                                    ##author ids
                                    author_id_placeholder = ''
                                    if 'ids' in author.keys() and author['ids'] != []: 
                                        for id_dict in author['ids']:
                                            if 'magid' in id_dict.values():
                                                author_id_placeholder = id_dict['value']
                                                break
                                        
                                    author_ids.append(author_id_placeholder)

                                    ##afilliations
                                    affiliation_placeholder = ''
                                    affiliation_ids_placeholder = ''
                                    country_placeholder = ''
                                    if 'affiliations' in author.keys() and author['affiliations'] != []:
                                        affiliation_placeholder = author['affiliations'][0]['name']
                                        
                                        #affiliation ids
                                        if 'ids' in author['affiliations'][0]:
                                            for id_dict in author['affiliations'][0]['ids']:
                                                if 'magid' in id_dict.values():
                                                    affiliation_ids_placeholder = id_dict['value']
                                                break
                                            

                                        #countries
                                        if 'country_code' in author['affiliations'][0].keys():
                                            country_placeholder = author['affiliations'][0]['country_code']

                                    affiliations.append(affiliation_placeholder)
                                    affiliations_ids.append(affiliation_ids_placeholder)
                                    affiliations_countries.append(country_placeholder)
                                    n = n+1
                                

# export it as dictionary
dict_author_edges = {'pubmed_id': pubmed_ids, 'titles': titles, 'year_published': dates, 'lens_scholarly_id': lens_scholarly_id, 'times_cited_sci': nr_of_times_cited_science, 'times_cited_pat': nr_of_times_cited_patents, 'affiliation_country' : affiliations_countries, 'affiliation': affiliations, 'affiliation_id': affiliations_ids, 'author_lastname' : authors_lastname, 'author_firstname' : authors_firstname, 'author_id' : author_ids,  'first_author': first_author, 'last_author': last_author} 
df_pubmedID_dict = pd.DataFrame(dict_author_edges)

In [18]:
len(set(df_pubmedID_dict['pubmed_id']))

23417

In [14]:
# why am I missing 21? 
comparl = list(set(df_pubmedID_dict['pubmed_id']))

#a = []
#for i in  l2: 
#    if i not in [int(x) for x in comparl]:
#        a.append(i)
#        print(i)

# 6 were because they had no authors, the other 15 pubmeds simply don't exist 

In [153]:
# look at unknonw countries to create rules below 
Counter(df_pubmedID_dict['affiliation'][df_pubmedID_dict['affiliation'].isin(still_missing)]).most_common()

[('', 5142),
 ('Affiliations can be found on Science Online (available at ).', 80),
 ('Fisheries Research Agency', 27),
 ('College of Fisheries', 27),
 ("All Children's Hospital", 19),
 ('Life Sciences School', 16),
 ('Department of Molecular and Cell Biology and the Center for Integrative Genomics,,',
  15),
 ('Crops Research Institute', 14),
 ('Animal and Comparative Sciences', 13),
 ('Olsen lab', 13),
 ('Department of Biology', 13),
 ('Marine Science Institute', 13),
 ('Center for Food Safety', 12),
 ('Abteilung Membranbiochemie', 11),
 ('Biochemistry and Molecular Biology', 11),
 ('Department of Biological Sciences', 11),
 ('Institute of Aquatic Economic Animals', 11),
 ('Biology', 10),
 ('Departamento de Oceanografia e Pescas', 10),
 ('SupAgro', 9),
 ('National Institute for Medical Research', 9),
 ('Biochemistry & Molecular Biology', 9),
 ('Departments of Microbiology', 9),
 ('RIKEN', 8),
 ('National Institute of Education', 8),
 ('Life Sciences Institute', 8),
 ('Integrative Bio

In [178]:
df_pubmedID_dict[df_pubmedID_dict['affiliation'] == "1Ministry of Education Key Laboratory for Biodiversity Science and Ecological Engineering, and Insti"]

,pubmed_id,titles,lens_scholarly_id,times_cited_sci,times_cited_pat,affiliation_country,affiliation,affiliation_id,author_lastname,author_firstname,author_id,first_author,last_author
127378,20064002,Cryptic Species and Historical Biogeography of...,082-263-853-687-983,20,,,1Ministry of Education Key Laboratory for Biod...,,Fu,Cuizhang,1981656715,not first_author,not last_author
127379,20064002,Cryptic Species and Historical Biogeography of...,082-263-853-687-983,20,,,1Ministry of Education Key Laboratory for Biod...,,Yin,Wei,2131240199,not first_author,not last_author
127380,20064002,Cryptic Species and Historical Biogeography of...,082-263-853-687-983,20,,,1Ministry of Education Key Laboratory for Biod...,,Li,Guo,2641490762,not first_author,not last_author
127381,20064002,Cryptic Species and Historical Biogeography of...,082-263-853-687-983,20,,,1Ministry of Education Key Laboratory for Biod...,,Chen,Hui,2687329430,not first_author,not last_author
127382,20064002,Cryptic Species and Historical Biogeography of...,082-263-853-687-983,20,,,1Ministry of Education Key Laboratory for Biod...,,Wu,Qianhong,2675121394,not first_author,not last_author
127383,20064002,Cryptic Species and Historical Biogeography of...,082-263-853-687-983,20,,,1Ministry of Education Key Laboratory for Biod...,,Li,Bo,3157847592,not first_author,last_author


In [179]:
for i in list(df_pubmedID_dict['affiliation'][df_pubmedID_dict['affiliation_country'] == '']):
    if  "Biotechnology Research Institute" in i:# and 'China' not in i: 
        print(i)
        

Biotechnology Research Institute, Chinese Academy of Agricultural Sciences, Beijing, China
Biotechnology Research Institute, Chinese Academy of Agricultural Sciences, Beijing, China
Biotechnology Research Institute, Chinese Academy of Agricultural Sciences, Beijing, China
Biotechnology Research Institute, Chinese Academy of Agricultural Sciences, Beijing, China
Biotechnology Research Institute, Chinese Academy of Agricultural Sciences, Beijing, China
Biotechnology Research Institute, Chinese Academy of Agricultural Sciences, Beijing, China
Biotechnology Research Institute, Chinese Academy of Agricultural Sciences, Beijing, China
Biotechnology Research Institute, Chinese Academy of Agricultural Sciences, Beijing, China
Environmental Biotechnology Department, Genetic Engineering and Biotechnology Research Institute, City of Scientific Research and Technology Applications, Alexandria, Egypt.
Bioprocess Development Department, Genetic Engineering and Biotechnology Research Institute, City 

In [5]:
# add condition if country is known in another entry, then label with existing info
# re-construct institutional countries based on their names, this is pre-cleaning before manual curation
affiliations = []
found_acountries = []
still_missing = []


count = 1
for affiliation in list(df_pubmedID_dict['affiliation'][df_pubmedID_dict['affiliation_country'] == '']): # in case of filtering for first authors & (df_pubmedID_dict['first_author'] == 'first_author')]) 
    ph = ''
    affiliations.append(affiliation)
    #print('\n\n'+str(count)) 
    #print(affiliation)
    

    if str(affiliation) != '': #some are just entirely unknown
        for country in list(pycountry.countries):
            if country.name in affiliation and 'Indiana University' not in affiliation and 'New Mexico' not in affiliation and 'New Jersey' not in affiliation:
                ph = pc.country_name_to_country_alpha2(country.name)
                
            if str(ph) == '':
                
                try: 
                    if country.common_name in affiliation: 
                        ph = pc.country_name_to_country_alpha2(country.name)
                        print('AHHH')
                        print('\n\n'+str(count)) 
                        print(affiliation)
                except: 
                    pass
                
        if str(ph) == '': 
            if ', UK ' in affiliation or ', UK,' in affiliation or 'London, England' in affiliation or 'Stevenage, England' in affiliation or 'UCL Institute of Ophthalmology' in affiliation or 'Strathclyde Institute of Pharmacy and Biomedical Sciences' in affiliation or 'Smith, Kline & French' in affiliation or 'Edward Jenner Institute for Vaccine Research' in affiliation or 'Department of Zoology;' in affiliation or 'Addenbrooke' in affiliation or ',UK' in affiliation or 'U.K.' in affiliation or 'Aberdeen' in affiliation or 'The Roslin Institute' in affiliation or 'Wellcome/CRC Institute for Developmental Biology and Cancer Research' in affiliation or 'Harwell Science and Innovation Campus' in affiliation or ', UK.' in affiliation or ', UK' in affiliation: 
                ph = 'GB'
            elif ', USA' in affiliation or 'Broad Institute of Massachusetts' in affiliation or 'Hobart Corporation' in affiliation or 'Departments of Civil and Environmental Engineering' in affiliation or 'Departments of aCivil and Environmental Engineering and' in affiliation or 'Procter & Gamble' in affiliation or 'Human Genome Sequencing Center' in affiliation or 'University of Oregon' in affiliation or 'University of Texas' in affiliation or 'Verenium Corporation' in affiliation or "Department of Medicine, Brigham and Women's Hospital" in affiliation or 'Edgewood Chemical Biological Center' in affiliation or 'Genome Science Group, Bioscience Division, Los Alamos National Laboratory, Los Alamos, NM 87545; and' in affiliation or 'Forsyth' in affiliation or 'Animal and Comparative Sciences' in affiliation or 'Department of Molecular and Cell Biology and the Center for Integrative Genomics' in affiliation or "All Children's Hospital" in affiliation or 'University of Rhode Island' in affiliation or 'Pacific Biosciences' in affiliation or 'International Sleep Products Association' in affiliation or 'Defenders of Wildlife' in affiliation or 'Texas Tech University' in affiliation or 'Pasadena, CA' in affiliation or 'Cleveland Clinic Lerner Research Institute' in affiliation or 'RI02809 USA' in affiliation or 'The Institute for Genomic Research, Rockville' in affiliation or 'Kosan Biosciences' in affiliation or 'National Jewish Center for Immunology and Respiratory Medicine' in affiliation or 'Sanford-Burnham Institute for Medical Research' in affiliation or 'La Jolla, CA' in  affiliation or 'University of Maine' in affiliation or 'Delaware Biotechnology Institute' in affiliation or 'Hawai' in affiliation or 'City University of New York' in affiliation or 'Invitrogen' in affiliation or 'University Hospital, Boston' in affiliation or 'National Museum of Natural History, Smithsonian Institution' in affiliation or 'Florida State University' in affiliation or 'New York State Psychiatric Institute' in affiliation or 'Pennsylvania' in affiliation or 'Roche Institute of Molecular Biology' in affiliation or 'Virginia Bioinformatics Institute' in affiliation or 'Departments of Microbiology and Thermal Biology Institute and' in affiliation or 'Nereus Pharmaceuticals' in affiliation or 'TigerLogic' in affiliation or 'From the Columbia River Inter-Tribal Fish Commission' in affiliation or 'College of Oceanic and Atmospheric Sciences, and' in affiliation or 'Florida A&M University' in affiliation or 'University of Dayton' in affiliation or 'Saint Petersburg, FL' in affiliation or 'St Petersburg, FL' in affiliation or 'Departments of Civil and Environmental Engineering,' in affiliation or 'Limnology and Marine Sciences Program' in affiliation or 'Salt Lake City' in affiliation or 'La Jolla, California' in affiliation or 'The Field Museum, Chicago' in affiliation or 'Icos' in affiliation or 'Ocean Genome Legacy, Inc' in affiliation or 'National Center for Agricultural Utilization Research' in affiliation or 'Division of Infectious Diseases and Department of Internal Medicine' in affiliation or 'Hospital Oakland Research Institute' in affiliation or 'La jolla, California' in affiliation or 'Yale University' in affiliation or 'University of Wisconsin' in affiliation or 'Maryland' in affiliation or 'Harvard University' in affiliation or 'University of Illinois' in affiliation or 'University of Washington' in affiliation or 'Stanford University' in affiliation or 'University of Chicago' in affiliation or 'Kewalo Marine Laboratory' in affiliation or 'University of California' in affiliation or 'Virginia Institute of Marine Science' in affiliation or 'University of Southern California' in affiliation or 'University of Pittsburgh' in affiliation or 'Columbia University' in affiliation or 'California Institute of Technology' in affiliation or 'University of Colorado' in affiliation or 'Craig Venter Institute' in affiliation or 'Josephine Bay Paul Center' in affiliation or 'Oregon State University' in affiliation or 'Maryland Department of Natural Resources Police' in affiliation or 'New Jersey' in affiliation or 'Palo Alto, California' in affiliation or 'University of Massachusetts' in affiliation or 'University of Miami School of Medicine' in affiliation or 'USA.' in affiliation or 'University of South Florida' in affiliation or 'Fox Chase Cancer Center' in affiliation or 'University of Pennsylvania' in affiliation or 'Chesapeake Biological Laboratory' in affiliation or 'Southampton' in affiliation or 'Washington University' in affiliation or 'Helen Wills Neuroscience Institute' in affiliation or 'California State University' in affiliation or 'Rosenstiel School of Marine and Atmospheric Science' in affiliation or 'US Department of Energy Joint Genome Institute' in affiliation or 'Monterey Bay Aquarium Research Institute' in affiliation or 'U. S. A.' in affiliation or 'U.S.A.' in affiliation or 'Kalamazoo College' in affiliation or 'Whitney Laboratory for Marine Bioscience' in affiliation or 'Baylor College of Medicine' in affiliation or 'Benaroya Research Institute' in affiliation or 'San Diego' in affiliation or 'Washington State University Vancouver' in affiliation or 'Oklahoma State University–Stillwater' in affiliation or 'University of Hawaii' in affiliation or 'Drexel University' in affiliation or 'University of Maryland' in affiliation or 'Worcester Foundation for Biomedical Research' in affiliation or 'Indiana University' in affiliation or 'National Cancer Institute, Frederick' in affiliation: 
                ph = 'US'
            elif ', Russia' in affiliation or 'Russian' in affiliation or 'Pacific Institute' in affiliation or 'Petersburg Nuclear Physics Institute' in affiliation or 'SibEnzyme' in affiliation or 'Российской академии наук' in affiliation or 'Virginia Institute of Marine Science' in affiliation or 'Moscow' in affiliation or 'All-Union Research Scientific Institute' in affiliation: 
                ph = 'RU'
            elif 'Republic of Korea' in affiliation or 'East Sea Fisheries Research Institute' in affiliation or 'Ulsan University' in affiliation or ', Korea' in affiliation or 'Yosu National University' in affiliation or 'Korea Polar Research Institute' in affiliation or 'Chungbuk University' in affiliation or 'Korea.' in affiliation or 'National Fisheries Research & Development Institute' in affiliation:
                ph = 'KR'
            elif ' Finnish Food Safety Authority' in affiliation: 
                ph = 'FI'
            elif 'Uiversitat Munchen' in affiliation or 'Developmental Biology Unit,,' in affiliation or 'Merck & Co' in affiliation or 'Abteilung Membranbiochemie' in affiliation or 'Albrecht-von-Haller-Institut' in affiliation or 'Kieler Wirkstoff-Zentrum am IFM-GEOMAR' in affiliation or 'Techn. Universitat Munchen' in affiliation or 'GSF—National Research Center for Environment and Health' in affiliation or 'Christians Albrechts Universität' in affiliation or 'Technische Universität, München' in affiliation or 'Institute of Integrative Biology (IBZ)' in affiliation or 'Lüneburg University' in affiliation or 'Heidelberg' in affiliation or 'Forschungsinstitut für die Biologie landwirtschaftlicher Nutztiere' in affiliation or 'Botanisches Institut' in affiliation or 'University Hamburg' in affiliation or 'Helmholtz Center for Infection Research' in affiliation: 
                ph = 'DE'
            elif 'Facultad de Ciencias Exactas y Naturales' in affiliation or 'Buenos Aires' in affiliation: 
                ph = 'AR'
            elif 'UPRRP College of Natural Sciences' in affiliation: 
                ph = 'PR'
            elif 'University of Tromsø' in affiliation or 'Unifob Petroleum' in affiliation or 'Bergen' in affiliation or 'Hedmark University College' in affiliation or 'University of Nordland' in affiliation or 'Norwegian' in affiliation:
                ph = 'NO'
            elif 'Pierre-and-Marie-Curie University' in affiliation or 'Ecologie, Evolution, Symbiose' in affiliation or 'aCentre National de la Recherche Scientifique, Unité Mixte de Recherche 7232 and' in affiliation or "Institut méditerranéen de biodiversité et d'écologie marine et continentale" in affiliation or 'SupAgro' in affiliation or 'Modèles en biologie cellulaire et évolutive' in affiliation or 'Muséum National d’Histoire Naturelle' in affiliation or 'SAMU Social' in affiliation or 'Institute for the Protection and Security' in affiliation or 'France FR' in affiliation or 'Station de Biologie Marine du Muséum National d’Histoire Naturelle' in affiliation or 'Université Pierre' in affiliation or 'UMR 958' in affiliation or 'Marion Merrell Dow' in affiliation or 'Equipe Ecophysiologie: Adaptation et Evolution Moléculaires' in affiliation or 'Institut de Génétique et Microbiologie and' in affiliation or 'Écologie et Paléontologie' in affiliation or 'Développement et évolution ' in affiliation or 'Adria Airways' in affiliation or 'ANSES' in affiliation or 'Blaise Pascal University' in affiliation or 'Nancy-Université' in affiliation or 'university of lille' in affiliation or 'Laboratoire de Bactériologie' in affiliation or 'University of Provence' in affiliation or 'Université de Pau' in affiliation or 'University of the Mediterranean' in affiliation or 'Biologie des Organismes et Ecosystèmes Aquatiques' in affiliation or 'University of Rennes' in affiliation or 'Mer et santé' in affiliation or 'Villefranche-sur-mer' in affiliation:
                ph = 'FR'
            elif 'Monash University' in affiliation or 'Museum of Tropical Queensland' in affiliation or 'Cooperative Research Centre' in affiliation or 'Queensland Department of Primary Industries' in affiliation or 'Queensland Biosciences Precinct' in affiliation: 
                ph = 'AU'
            elif 'México' in affiliation or 'CINVESTAV' in affiliation or 'Escuela Nacional de Ciencias Biológicas (ENCB)' in affiliation or 'Universidad Autónoma de Yucatán' in affiliation or 'Centro de Investigación en Alimentación y Desarrollo' in affiliation or 'Centro de Investigacion en Alimentacion y Desarrollo' in affiliation: 
                ph = 'MX'
            elif 'Japan, JP' in affiliation or '*Biochemistry and' in affiliation or 'Kazusa DNA Research Institute' in affiliation or 'Department of Bacterial Infections1 and' in affiliation or '*Subground Animalcule Retrieval Program and' in affiliation or 'Gulf Coast Regional Blood Center' in affiliation or 'Fisheries Research Agency' in affiliation or 'Marine Biotechnology Institute (MBI)' in affiliation or 'RIKEN' in affiliation or 'Yamada Junior College of Home Economics' in affiliation or 'Amase Institute, Hayashibara Biochemical' in affiliation or 'Subground Animalcule Retrieval (SUGAR) Project' in affiliation or 'Suntory Institute for Bioorganic Research' in affiliation or 'Hokkaido Universit' in affiliation or 'National Research Institute of Fisheries Science, Yokohama' in affiliation or 'Okazaki Institute for Integrative Bioscience' in affiliation or 'Nichirei Corporation' in affiliation or 'Institute for Fermentation' in affiliation or 'Hitec' in affiliation or 'Saga Group' in affiliation or 'Nihon Shokuhin Kako' in affiliation or 'Laboratory of QC and Food Analysis' in affiliation or 'Division of Medical Genetics and Biochemistry' in affiliation or 'Mitsubishi' in affiliation or 'National Presto Industries' in affiliation or 'Tsukuba' in affiliation or 'Osaka' in affiliation or 'UniversityKochi' in affiliation or 'Sophia UniversityTokyo' in affiliation or 'Nippon Suisan Kaisha' in affiliation or 'Suntory Limited Research Center' in affiliation or 'Nagasaki Prefectural Institute of Fisheries' in affiliation or 'Tohoku UniversitySendai' in affiliation or 'National Research Institute of Fisheries and Environment of Inland Sea' in affiliation or 'Tohoku National Fisheries Research Institute' in affiliation or 'Iwate' in affiliation or 'Shinagawa Aquarium' in affiliation or 'Miyazaki University' in affiliation or 'Subground Animalcule Retrieval (SUGAR) Program' in affiliation: 
                ph = 'JP'
            elif 'National Institute of Education' in affiliation: 
                ph = 'SG'
            elif 'University of the Algarve' in affiliation or 'Português' in affiliation or 'Departamento de Oceanografia e Pescas' in affiliation or 'Universidade Nova de Lisboa' in affiliation or 'Instituto de Biologia Molecular e Celular' in affiliation: 
                ph = 'PT'
            elif 'Halifax' in affiliation or 'NRC Institute for Marine Biosciences' in affiliation or 'BC Research Inc' in affiliation or 'Université du Québec' in affiliation or 'Université du Québec à Montréal' in affiliation or 'Marine Institute of Memorial University of Newfoundland' in affiliation: 
                ph = 'CA'
            elif 'Östersidan' in affiliation: 
                ph = 'SE'
            elif 'Andrés Bello National University' in affiliation or 'Life Sciences School' in affiliation or 'Fundación Ciencia para la Vida' in affiliation or 'Pontifical Catholic University of Valparaíso' in affiliation: 
                ph = 'CL'
            elif 'Basel Institute for Immunology' in affiliation or 'Genentech' in affiliation or 'Swiss Institute of Allergy and Asthma Research' in affiliation or 'ISREC' in affiliation or 'Zürich' in affiliation or 'Friedrich Miescher Institute for Biomedical Research' in affiliation: 
                ph = 'CH'
            elif 'Sathabama University' in affiliation or 'National Centre for Biological Sciences' in affiliation or 'National Research Centre on Plant Biotechnology, LBS Building, Pusa Campus' in affiliation or 'Thiagarajar College' in affiliation or 'Stella Maris College' in affiliation or 'Calcutta' in affiliation or 'Nalco Holding Company' in affiliation or 'Kolkata' in affiliation or 'National Centre for Antarctic Ocean Research' in affiliation: 
                ph = 'IN'
            elif 'Institute of Business & Medical Careers' in affiliation or 'ANFACO-CECOPESCA' in affiliation or 'Centro de Investigacións Mariñas' in affiliation or 'University of Castilla' in affiliation or 'Generalitat of Catalonia' in affiliation or 'Área de Microbiología' in affiliation or 'Instituto Potosino de Investigación Científica y Tecnológica' in affiliation or 'Universidad Miguel Hernández de Elche' in affiliation or 'ICM Partners' in affiliation or 'Miguel Hernández University' in affiliation or 'Instituto Español de Oceanografía' in affiliation or 'España' in affiliation or 'Instituto Nacional de Técnica Aeroespacial' in affiliation: 
                ph = 'ES'
            elif 'Catholic University of Leuven' in affiliation or 'University of Hasselt' in affiliation or 'Free University of Brussels' in affiliation or 'Université de Namur' in affiliation: 
                ph = 'BE'
            elif 'Czech Republic' in affiliation or 'Prague' in affiliation: 
                ph = 'CZ'
            elif 'Fondazione Don Carlo Gnocchi Onlus' in affiliation or 'CONISMA ULR' in affiliation or 'Seconda Università degli Studi di Napoli' in affiliation or 'Università di Napoli' in affiliation or 'Pozzuoli (Napoli)' in affiliation or 'IRSA' in affiliation or 'Istituto di Biomedicina' in affiliation or 'Italia' in affiliation or 'CNR-ISE Istituto per lo Studio degli Ecosistemi' in affiliation or 'CNR-ICVBC Istituto per la Conservazione e la Valorizzazione dei Beni Culturali' in affiliation or 'Istituto di Biomedicina e Immunologia Molecolare “Alberto Monroy”' in affiliation or 'University of Genova' in affiliation or 'Istituto Nazionale per la Ricerca sul Cancro' in affiliation: 
                ph = 'IT'
            elif 'Evandro Chagas Institute' in affiliation or 'Ribeirão Preto' in affiliation or 'Brasil.' in affiliation or 'LABOMAR' in affiliation or 'Laboratório de Genética Microbiana' in affiliation or 'UNIVALI' in affiliation or 'Federal University of Ceará' in affiliation or 'Empresa Brasileira' in affiliation or 'Pontifícia Universidade Católica do Rio Grande do Sul' in affiliation: 
                ph = 'BR' 
            elif 'Seibersdorf' in affiliation or 'Universität für Bodenkultur' in affiliation: 
                ph = 'AT'
            elif 'Kathmandu' in affiliation: 
                ph = 'NP'
            elif 'University of Ouargla' in affiliation or 'Bejaia, Algérie' in affiliation: 
                ph = 'DZ'
            elif 'Xiamen University' in affiliation or '1Ministry of Education Key Laboratory for Biodiversity Science and Ecological Engineering, and Insti' in affiliation or 'CAS - Institute of Genetics and Developmental Biology' in affiliation or 'Crops Research Institute' in affiliation or 'Institute of Aquatic Economic Animals' in affiliation or 'College of Fisheries' in affiliation or 'Tianjin Medical College' in affiliation or 'Liaoning Ocean and Fisheries Science Research Institute' in affiliation or 'Southern University of Science and Technology' in affiliation or 'State Key Laboratory Breeding Base of Marine Genetic Resources' in affiliation or 'Yunnan Agriculture University' in affiliation or 'Nankai University' in affiliation: 
                ph = 'CN'
            elif 'Danish' in affiliation or 'danish' in affiliation or 'Palsbøll lab' in affiliation or 'Royal Veterinary and Agricultural University' in affiliation or 'Odense University' in affiliation: 
                ph = 'DK'
            elif 'Finnish' in affiliation: 
                ph = 'FI'
            elif 'Iran' in affiliation:
                ph = 'IR'
            elif 'University of Port Elizabeth' in affiliation: 
                ph = 'ZA' 
            elif 'University of Manouba' in affiliation or 'Sfax, Tunisie' in affiliation: 
                ph = 'TN'
            elif 'Panamá' in affiliation:
                ph = 'PA'
            elif 'Tunceli University' in affiliation: 
                ph = 'TK'
            elif 'Perú' in affiliation: 
                ph = 'PE'
            elif 'Departments of aEarth and Planetary Sciences,' in affiliation:
                ph = 'IL'
            elif 'Hung Kuang Institute of Technology' in affiliation or 'Biodiversity Research Center' in affiliation: 
                ph = 'TW'
            elif 'Industrial Research Limited' in affiliation or 'Auckland War Memorial Museum' in affiliation or 'Bay of Plenty Polytechnic' in affiliation:  
                ph = 'NZ'
            elif 'Van Elsas lab' in affiliation or 'Olsen lab' in affiliation or 'Naturalis' in affiliation or 'Centraalbureau voor Schimmelcultures' in affiliation or 'Leiden' in affiliation or 'Wageningen Agricultural University' in affiliation: 
                ph = 'NL'
            elif 'University of Science and Technology of Hanoi' in affiliation: 
                ph = 'VN'
            elif 'Taylors University' in affiliation: 
                ph = 'MY'
            elif 'Grupo de Genética, Centro Nacional de' in affiliation: 
                ph = 'CU'
            elif 'Reykjavik' in affiliation: 
                ph = 'IS'
            elif 'Nicolaus Copernicus University' in affiliation: 
                ph = 'PL'
            elif 'Foundation for Research & Technology – Hellas' in affiliation: 
                ph = 'GR'
            elif 'Ruđer Bošković Institute' in affiliation: 
                ph = 'HR'
            elif 'Hungarian' in affiliation: 
                ph = 'HU'
            else: 
                #if the same affiliaiton has a country in another datapoint and if those countries are not conflicting, label affiliation with the known country info
                if len(df_pubmedID_dict[(df_pubmedID_dict['affiliation'] == affiliation) & (df_pubmedID_dict['affiliation_country'] != '')]) > 0 and len(set(df_pubmedID_dict['affiliation_country'][(df_pubmedID_dict['affiliation'] == affiliation) & (df_pubmedID_dict['affiliation_country'] != '')])) > 0:
                    ph = list(set(df_pubmedID_dict['affiliation_country'][(df_pubmedID_dict['affiliation'] == affiliation) & (df_pubmedID_dict['affiliation_country'] != '')]))[0]
                
                # else leave it 
                else: 
                    still_missing.append(affiliation)
    
    #labl as still missing if they do not even have an affiliation reported
    else: 
        still_missing.append(affiliation)
    
    found_acountries.append(ph)
    
    count = count +1
    


AHHH


24
Freshwater Aquaculture Research Center, Fisheries Research Institute, Lukang 505, Taiwan
AHHH


25
Tungkang Biotechnology Research Center, Fisheries Research Institute, Pintung 944, Taiwan
AHHH


154
Department of Aquatic Biosciences, National Chyai University, Chyai City 60004, Taiwan.
AHHH


300
RCOID and Department of Biotechnology and Bioengineering, Pusan, South Korea.
AHHH


301
RCOID and Department of Biotechnology and Bioengineering, Pusan 608-737, South Korea
AHHH


302
RCOID and Department of Biotechnology and Bioengineering, Pusan 608-737, South Korea
AHHH


371
Korea Basic Science Institute, Anam-Dong Sungbuk-Gu, Seoul, South Korea
AHHH


466
Polar BioCenter, Korea Polar Research Institute, Korea Ocean Research and Development Institute, Incheon 406-840, South Korea
AHHH


481
Institute of Biotechnology, National Center for Natural Science and Technology, Nghia do, Tu liem, Hanoi, Vietnam
AHHH


482
Institute of Biotechnology, National Center for Natural Science a

AHHH


5364
Korea Polar Research Institute, Songdo Techno Park, Incheon 406-840, South Korea
AHHH


5563
Polar BioCenter, Korea Polar Research Institute, Korea Ocean Research and Development Institute, Incheon 406-840, South Korea
AHHH


5564
Faculty of Marine Bioscience and Technology, College of Life Sciences, Kangnung-Wonju National University, Gangneung 210-702, South Korea
AHHH


5567
d WeThink Biotech INC. , Taoyuan , Taiwan.
AHHH


5734
Department of Planning and Research, National Museum of Marine Biology and Aquarium, Pingtung 944, Taiwan, ROC
AHHH


5735
Department of Planning and Research, National Museum of Marine Biology and Aquarium, Pingtung 944, Taiwan, ROC
AHHH


5933
Division of Life Sciences, Korea Polar Research Institute, Incheon 406-840, South Korea.
AHHH


5934
Division of Life Sciences, Korea Polar Research Institute, Incheon 406-840, South Korea.
AHHH


6081
d WeThink Biotech Inc. , Taoyuan , Taiwan , and.
AHHH


6172
Institute of Biotechnology, National Center

AHHH


10954
Department of Biochemical Science and Technology, NTU, Taipei 106, Taiwan, ROC
AHHH


11027
Ewha Womans University, Division of EcoScience, Seoul, South Korea
AHHH


11161
Taiwan Ocean Research Institute of the National Applied Research Laboratories
AHHH


11203
National Museum of Marine Biology and Aquarium, Checheng, Pingtung, Taiwan.
AHHH


11204
National Museum of Marine Biology and Aquarium, Checheng, Pingtung, Taiwan.
AHHH


11230
Tungkang Biotechnology Research Center, Fisheries Research Institute, Tungkang, 20246, Taiwan.
AHHH


11617
Taiwan Fisheries Research Institute, 199 Hou-Ih Road, Keelung, Taiwan, ROC
AHHH


11618
National Museum of Marine Biology and Aquarium, Checheng, Pingtung, Taiwan, ROC
AHHH


11742
b Animal Resources Division , National Institute of Biological Resources , Incheon , South Korea , and.
AHHH


12951
Department of Fisheries Production and Management, National Kaohsiung University of Science and Technology, Kaohsiung, Taiwan.
AHHH


12976


AHHH


18487
Division of Food Science and Biotechnology, College of Fishery Sciences, Pukyung National University, Pusan 608-737, South Korea
AHHH


18607
Department of Biotechnology, Sangmyung University, Seoul, 03016, South Korea.
AHHH


18608
Department of Biotechnology, Sangmyung University, Seoul, 03016, South Korea.
AHHH


18609
Department of Biotechnology, Sangmyung University, Seoul, 03016, South Korea. evodevo@smu.ac.kr.
AHHH


18764
WeThink Biotech INC., Taoyuan, Taiwan, 
AHHH


18840
Tanzania Fisheries Research Institute, P.O. Box 9750, Dar es Salaam, Tanzania;
AHHH


18841
Tanzania Fisheries Research Institute, P.O. Box 9750, Dar es Salaam, Tanzania;
AHHH


18961
Aquaculture Breeding Institute Hualien, Hualien 97445, Taiwan, ROC
AHHH


19100
d WeThink Biotech Inc. , Taoyuan, Taiwan , ROC , and.
AHHH


19288
Department of Aquatic Life Medicine, PKNU, Busan 608-737, South Korea
AHHH


19561
Department of Medical Research, Chi Mei Medical Center, Tainan, Taiwan, ROC
AHHH


202

AHHH


25038
National Museum of Marine Biology and Aquarium, Pintung 944, Taiwan
AHHH


25208
Department of Food and Nutrition, Hung Kuang University, Sha Lu, Taiwan.
AHHH


25332
d WeThink Biotech INC. , Taoyuan , Taiwan .
AHHH


25346
b National Museum of Marine Biology and Aquarium , Pingtung , Taiwan .
AHHH


25347
d WeThink Biotech Inc. , Taoyuan , Taiwan , and.
AHHH


25458
Aquaculture Environmental Institute, NFRDI, Yongun‐ri Sanyang‐up, Tongyeong City, Kyeongsangnam‐do 650‐943, South Korea
AHHH


25459
Faculty of Marine Bioscience and Technology, Kangnung National University, Gangneung 210‐702, South Korea
AHHH


25553
4Tanzania Fisheries Research Institute, Dar es Salaam, Tanzania;
AHHH


25554
4Tanzania Fisheries Research Institute, Dar es Salaam, Tanzania;
AHHH


25555
5Regional Commissioner's Office Tanga, Tanga, Tanzania;
AHHH


25628
Korea Polar Research Institute, Songdo Techno Park, Incheon 406-840, South Korea
AHHH


25629
Korea Polar Research Institute, Songdo Techno 

In [16]:
# How many could I reconstruct out of the x missing?
len([x for x in found_acountries if x != ''])

21824

In [17]:
len(set(still_missing))

1151

In [27]:
# create new dataframe to include the newly found
df2 = df_pubmedID_dict[df_pubmedID_dict['affiliation_country'] == '']
df2['affiliation_country'] = found_acountries

# merge both 
df3 = pd.merge(df_pubmedID_dict[df_pubmedID_dict['affiliation_country'] != ''],df2, how = 'outer').rename(columns = {'affiliation_country':'affiliation_country_ed'}).sort_values(['pubmed_id', 'author_lastname', 'author_firstname'], ascending = False) # add those that already had country to those that have a new country

# remerge the original affiliation countries (just for comparison)
df3['affiliation_country'] = list(df_pubmedID_dict.sort_values(['pubmed_id', 'author_lastname', 'author_firstname'], ascending = False)['affiliation_country'])


<ipython-input-27-5c12aa9fa682>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['affiliation_country'] = found_acountries


In [184]:
# merge universities if necessary
from collections import Counter
Counter(list(df3['affiliation'])).most_common()

[('', 5142),
 ('Chinese Academy of Sciences', 3336),
 ('Centre national de la recherche scientifique', 1650),
 ('University of Tokyo', 1605),
 ('Ocean University of China', 1573),
 ('Spanish National Research Council', 1380),
 ('Chinese Academy of Fishery Sciences', 1242),
 ('Sun Yat-sen University', 1240),
 ('Max Planck Society', 1030),
 ('Kyoto University', 794),
 ('Jeju National University', 729),
 ('Xiamen University', 693),
 ('Hokkaido University', 640),
 ('University of Bergen', 616),
 ('University of California, San Diego', 608),
 ('Joint Genome Institute', 594),
 ('Shandong University', 592),
 ('Tokyo University of Marine Science and Technology', 542),
 ('Russian Academy of Sciences', 533),
 ('IFREMER', 526),
 ('Zhejiang Ocean University', 500),
 ('University of Queensland', 499),
 ('University of Washington', 482),
 ('University of Aberdeen', 474),
 ('National Fisheries Research & Development Institute', 467),
 ('University of Mainz', 451),
 ('University of Barcelona', 432),
 

In [20]:
# add another round of country affiliations where necessary --> no not necessary for now
from collections import Counter
Counter(still_missing).most_common()

[('Affiliations can be found on Science Online (available at ).', 80),
 ('Fisheries Research Agency', 27),
 ('College of Fisheries', 27),
 ("All Children's Hospital", 19),
 ('Life Sciences School', 16),
 ('Department of Molecular and Cell Biology and the Center for Integrative Genomics,,',
  15),
 ('Crops Research Institute', 14),
 ('Animal and Comparative Sciences', 13),
 ('Olsen lab', 13),
 ('Department of Biology', 13),
 ('Marine Science Institute', 13),
 ('Center for Food Safety', 12),
 ('Abteilung Membranbiochemie', 11),
 ('Biochemistry and Molecular Biology', 11),
 ('Department of Biological Sciences', 11),
 ('Institute of Aquatic Economic Animals', 11),
 ('Biology', 10),
 ('Departamento de Oceanografia e Pescas', 10),
 ('SupAgro', 9),
 ('National Institute for Medical Research', 9),
 ('Biochemistry & Molecular Biology', 9),
 ('Departments of Microbiology', 9),
 ('RIKEN', 8),
 ('National Institute of Education', 8),
 ('Life Sciences Institute', 8),
 ('Integrative Biology', 8),
 (

In [15]:
# how many countries?
len(set(df3['affiliation_country_ed'][df3['affiliation_country_ed'] != '']))

150

In [14]:
# how many affiliations?  
len(set(df3['affiliation'][df3['affiliation'] != '']))

12589

In [13]:
# how many authors? 
df3[['author_lastname', 'author_firstname']].drop_duplicates()

,author_lastname,author_firstname
111431,Struck,Torsten H
111433,Hoesel,Christoph
111432,Golombek,Anja
111435,Elgetany,Asmaa Haris
111434,Dimitrov,Dimitar
...,...,...
104614,Nazar,R.N.
109425,Jenkins,John R.
108890,Kato,Mayumi
103024,O,Isaac


In [12]:
# how many authorships? 
df3[['author_lastname', 'author_firstname']]

,author_lastname,author_firstname
111431,Struck,Torsten H
111433,Hoesel,Christoph
111432,Golombek,Anja
111435,Elgetany,Asmaa Haris
111434,Dimitrov,Dimitar
...,...,...
108890,Kato,Mayumi
75846,Takemura,Shosuke
75845,Komiya,Hiroyuki
103024,O,Isaac


In [11]:
# how many first authors? 
df3[['author_lastname', 'author_firstname', 'first_author']][df3['first_author'] == 'first_author'].drop_duplicates()

,author_lastname,author_firstname,first_author
111431,Struck,Torsten H,first_author
123818,Corner,Richard D,first_author
117741,Rendina,Francesco,first_author
121359,Hasan,Mohammad Eusuf,first_author
117487,Galea,Horia R,first_author
...,...,...,...
32587,Ecarot-Charrier,B.,first_author
17678,Wegnez,Maurice,first_author
104614,Nazar,R.N.,first_author
109425,Jenkins,John R.,first_author


In [10]:
# how many first authors? 
df3[['author_lastname', 'author_firstname', 'last_author']][df3['last_author'] == 'last_author'].drop_duplicates()

,author_lastname,author_firstname,last_author
111435,Elgetany,Asmaa Haris,last_author
123820,Cutmore,Scott C,last_author
117748,Russo,Giovanni Fulvio,last_author
121368,Durand,Jean-Dominique,last_author
117489,Galli,Paolo,last_author
...,...,...,...
87211,Smith,Hamilton O.,last_author
17681,Clerot,Jean-Claude,last_author
104615,Roy,K.L.,last_author
109425,Jenkins,John R.,last_author


In [35]:
#Dates of publication
Counter(df3[['year_published','pubmed_id']].drop_duplicates()['year_published']).most_common()

[(2013, 1236),
 (2011, 1233),
 (2014, 1209),
 (2012, 1204),
 (2009, 1194),
 (2010, 1189),
 (2008, 1137),
 (2006, 1084),
 (2015, 1067),
 (2007, 1051),
 (2004, 897),
 (2005, 841),
 (2016, 830),
 (2003, 740),
 (2017, 659),
 (2002, 642),
 (2018, 611),
 (2000, 576),
 (2001, 568),
 (2019, 547),
 (1999, 504),
 (2021, 496),
 (2020, 480),
 (1998, 429),
 (1997, 407),
 (2022, 353),
 (1996, 351),
 (1995, 307),
 (1994, 295),
 (1993, 280),
 (1992, 213),
 (1991, 148),
 (1990, 111),
 (1989, 109),
 (1988, 95),
 (1987, 62),
 (2023, 49),
 (1986, 48),
 (1983, 39),
 (1985, 39),
 (1984, 30),
 (1982, 19),
 (1981, 14),
 (1980, 8),
 (1979, 5),
 (1975, 3),
 (1978, 3),
 (1976, 2),
 (1967, 1),
 (1974, 1),
 (1977, 1)]

In [38]:
# comparesd to dates of sequence entries
Counter(df[['ANs','Dates']]['Dates']).most_common()

[(2014, 6647098),
 (2021, 6359924),
 (2017, 3499275),
 (2011, 2777512),
 (2015, 2340004),
 (2008, 2042917),
 (2013, 1853602),
 (2010, 1700542),
 (2018, 1387823),
 (2012, 1289613),
 (2019, 1178727),
 (2007, 1163737),
 (2016, 949814),
 (2009, 819180),
 (2022, 807543),
 (2006, 607233),
 (2005, 380345),
 (2004, 324802),
 (2020, 299131),
 (2000, 193848),
 (2001, 79228),
 (2002, 77217),
 (2023, 45590),
 (2003, 15726),
 (1999, 3583),
 (1993, 1186),
 (1998, 1185),
 (1996, 1042),
 (1997, 960),
 (1995, 855),
 (1994, 478),
 (1992, 71),
 (1991, 32),
 (1989, 8),
 (1988, 6),
 (1990, 6),
 (1983, 5),
 (1987, 1),
 (1985, 1)]

In [39]:
manually_edited = []
for i in df3['affiliation']:
    if i in still_missing:
        manually_edited.append('unedited')
    if i not in still_missing:
        manually_edited.append('manually edited')

In [40]:
df3['edited_country'] = manually_edited

In [42]:
# export three different datasets: first authors
df3_firstauth = df3[df3['first_author'] == 'first_author']
# export the data files
df3_firstauth.to_csv(PATH\data\CoR-firstauth_Genbank[OBIS_and_patents].csv',index=False)

# export three different datasets: last author
df3_lastauth = df3[df3['last_author'] == 'last_author']
# export the data files
df3_lastauth.to_csv(PATH\data\CoR-lastauth_Genbank[OBIS_and_patents].csv',index=False)

# export three different datasets: all authors
df3.to_csv(PATH\data\CoR-allauth_Genbank[OBIS_and_patents].csv',index=False)